# calculate rotation and transposition matrix

In [34]:
import os

old_folder = r'D:\Pu\20211217-P_brain_M1_nonclear\Alignment'

before_position_file = os.path.join(old_folder, '10x_positions_before.txt')

new_folder = r'D:\Pu\20211226-P_brain_CTP11-1000_CTP12-DNA_from-1217\Alignment'

after_position_file = os.path.join(new_folder, '10x_positions_after.txt')

In [35]:
os.path.isfile(before_position_file), os.path.isfile(after_position_file)

(True, True)

In [36]:
import numpy as np
import os, sys
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [37]:
R, T = align_manual_points(before_position_file, after_position_file, save=False, save_folder=new_folder)


In [38]:
R, T

(array([[ 0.99999531, -0.00306106],
        [ 0.00306106,  0.99999531]]),
 array([-2338.22196778,   332.89137225]))

# transpose 60x positions

In [12]:

old_positions = np.loadtxt(os.path.join(os.path.dirname(old_folder), 'positions_all.txt'), delimiter=',')

In [13]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[-6.10145037e+03 -2.07574894e+03]
 [-6.10465801e+03 -1.87577466e+03]
 [-5.90789137e+03 -1.67259275e+03]
 [-5.90468373e+03 -1.87256702e+03]
 [-5.90147609e+03 -2.07254130e+03]
 [-5.89826846e+03 -2.27251558e+03]
 [-5.89506082e+03 -2.47248985e+03]
 [-5.69508655e+03 -2.46928222e+03]
 [-5.69829418e+03 -2.26930794e+03]
 [-5.70150182e+03 -2.06933366e+03]
 [-5.70470946e+03 -1.86935939e+03]
 [-5.70791709e+03 -1.66938511e+03]
 [-5.50794282e+03 -1.66617748e+03]
 [-5.50473518e+03 -1.86615175e+03]
 [-5.50152754e+03 -2.06612603e+03]
 [-5.49831991e+03 -2.26610030e+03]
 [-5.49511227e+03 -2.46607458e+03]
 [-5.29193036e+03 -2.66284122e+03]
 [-5.29513799e+03 -2.46286694e+03]
 [-5.29834563e+03 -2.26289267e+03]
 [-5.30155327e+03 -2.06291839e+03]
 [-5.30476090e+03 -1.86294411e+03]
 [-5.30796854e+03 -1.66296984e+03]
 [-5.31117618e+03 -1.46299556e+03]
 [-5.11120190e+03 -1.45978793e+03]
 [-5.10799426e+03 -1.65976220e+03]
 [-5.10478663e+03 -1.85973648e+03]
 [-5.10157899e+03 -2.05971075e+03]
 [-5.09837135e+03 -2

In [14]:
save_filename = os.path.join(new_folder, 'translated_positions_all.txt')
print(save_filename)
np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

D:\Pu\20211220-P_brain_CTP11-1000_CTP12-DNA_from_1216\Alignments\translated_positions_all.txt


# further adjust manually

In [17]:
before_adjust = np.array([[-6101.45,-2075.75],
                         [-4498.45,-2250.06],
                         [-5118.13,100.40],
                         ])


after_adjust = np.array([[-6060, -2062],
                        [-4455.0,-2237.0],
                        [-5072.2,115.5],
                        ])


after_adjust -before_adjust

array([[41.45, 13.75],
       [43.45, 13.06],
       [45.93, 15.1 ]])

In [18]:
manual_shift = np.mean(after_adjust -before_adjust, axis=0)
manual_shift

array([43.61, 13.97])

In [19]:
#manual_shift = np.array([-28.1,  -8.7])
adjusted_new_positions = new_positions + manual_shift

In [20]:
adj_save_filename = os.path.join(new_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

D:\Pu\20211220-P_brain_CTP11-1000_CTP12-DNA_from_1216\Alignments\adjusted_translated_positions_all.txt
